<a href="https://colab.research.google.com/github/BrotherKim/Colab/blob/main/SEP592/BK_Resnet50_Breast_classification_0616_fail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['ROOTDIR'] = "/content/jt_breast_aug"

In [3]:
!mkdir -p $ROOTDIR
!unzip /content/gdrive/MyDrive/breast_aug.zip -d $ROOTDIR

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/jt_breast_aug/TIL/train___TIL__TCGA-05-4417-01Z-00-DX1_azhao__2090_1_2.png  
  inflating: /content/jt_breast_aug/TIL/train___TIL__luad_batch_1__292_2_1.png  
  inflating: /content/jt_breast_aug/TIL/train___TIL__luad_batch_5__2524_1_2.png  
  inflating: /content/jt_breast_aug/TIL/train___TIL__luad_batch_1__593_2.png  
  inflating: /content/jt_breast_aug/TIL/test___TIL__TCGA-05-4396-01Z-00-DX1_azhao__580_2_1.png  
  inflating: /content/jt_breast_aug/TIL/test___TIL__luad_batch_1__645_2_2.png  
  inflating: /content/jt_breast_aug/TIL/test___TIL__TCGA-A2-A04Y-01Z-00-DX1_agreement__119_1_3.png  
  inflating: /content/jt_breast_aug/TIL/train___TIL__TCGA-A2-A04P-01Z-00-DX1_agreement__181_2.png  
  inflating: /content/jt_breast_aug/TIL/train___TIL__luad_batch_1__1561_2.png  
  inflating: /content/jt_breast_aug/TIL/valid___TIL__TCGA-A2-A04Y-01Z-00-DX1_agreement__18_2_1.png  
  inflating: /content/jt_breast_aug/TIL/train___TIL__luad_batch_

In [4]:
!ls -1 $ROOTDIR

Normal
TIL


In [5]:
!mkdir -p $ROOTDIR/train_set/
!mkdir -p $ROOTDIR/test_set/TIL
!mkdir -p $ROOTDIR/test_set/Normal
!mkdir -p $ROOTDIR/valid_set/TIL
!mkdir -p $ROOTDIR/valid_set/Normal

In [6]:
!cd $ROOTDIR/TIL && ls -1 | wc -l

48712


In [7]:
!cd $ROOTDIR/TIL && ls -1 | head -4800 | xargs -i mv "{}" $ROOTDIR/test_set/TIL

In [8]:
!cd $ROOTDIR/TIL && ls -1 | head -4800 | xargs -i mv "{}" $ROOTDIR/valid_set/TIL

In [9]:
!cd $ROOTDIR/TIL && ls -1 | wc -l

39112


In [10]:
!mv $ROOTDIR/TIL $ROOTDIR/train_set/TIL

In [11]:
!cd $ROOTDIR/Normal && ls -1 | wc -l

52888


In [12]:
!cd $ROOTDIR/Normal && ls -1 | head -5280 | xargs -i mv "{}" $ROOTDIR/test_set/Normal

In [13]:
!cd $ROOTDIR/Normal && ls -1 | head -5280 | xargs -i mv "{}" $ROOTDIR/valid_set/Normal

In [14]:
!cd $ROOTDIR/Normal && ls -1 | wc -l

42328


In [15]:
!mv $ROOTDIR/Normal $ROOTDIR/train_set/Normal

In [16]:
!cd $ROOTDIR/train_set/TIL && ls -1 | wc -l
!cd $ROOTDIR/train_set/Normal && ls -1 | wc -l
!cd $ROOTDIR/test_set/TIL && ls -1 | wc -l
!cd $ROOTDIR/test_set/Normal && ls -1 | wc -l
!cd $ROOTDIR/valid_set/TIL && ls -1 | wc -l
!cd $ROOTDIR/valid_set/Normal && ls -1 | wc -l

39112
42328
4800
5280
4800
5280


# Dataset 형태
datasets<br>
    &emsp;&emsp;└── test_set<br>
    &emsp;&emsp;│   &emsp;&emsp;├── class1<br>
    &emsp;&emsp;│   &emsp;&emsp;│   &emsp;&emsp;└── datas...<br>
    &emsp;&emsp;│   &emsp;&emsp;└── class2<br>
    &emsp;&emsp;│       &emsp;&emsp;&emsp;&emsp;&emsp;└── datas...<br>
    &emsp;&emsp;└── training_set<br>
        &emsp;&emsp;&emsp;&emsp;├── class1<br>
        &emsp;&emsp;&emsp;&emsp;│   &emsp;&emsp;└── datas...<br>
        &emsp;&emsp;&emsp;&emsp;└── class2<br>
            &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── datas...<br>


# Parameter 및 Basemodel 설정

In [17]:
IMAGE_SIZE = 224
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
NUM_CLASSES = 2
WEIGHT_DIR_PATH = '/content/gdrive/MyDrive/KAIST/SEP592/checkpoints'
WEIGHT_FILE_PATH = 'resnet50_brca_0616.h5'

In [18]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

baseModel = ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE, pooling='avg', classes=NUM_CLASSES)
#baseModel = ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE, classes=NUM_CLASSES)
#baseMode =ResNet50(include_top=True, weights=None, input_shape=INPUT_SHAPE, classes=NUM_CLASSES)

baseModel.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [19]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 위노그라드 알고리즘 설정
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

#rootPath = '/content/datasets'
#rootPath = '/content/gdrive/MyDrive/KAIST/SEP592/jt_breast_aug'
rootPath = os.environ['ROOTDIR']

imageGenerator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[.2, .2],
    horizontal_flip=True,
    validation_split=.1
)

In [20]:
#!find /content/gdrive/MyDrive/KAIST/lym_augmentation -type f | wc -l

In [21]:
#!cp -r /content/gdrive/MyDrive/KAIST/lym_aug1mentation/* /content/gdrive/MyDrive/KAIST/dataset_augmentation/train_set/lym/

In [22]:
#!find /content/gdrive/MyDrive/KAIST/dataset_augmentation/train_set/lym -type f | wc -l

In [23]:
#!cd /content/gdrive/MyDrive/KAIST/lym_aug1mentation/ && find ./ -name "*aug*" | sed -e 'p' -e "s/aug/aug1/g" |xargs -n 2 mv


In [24]:
#!find /content/gdrive/MyDrive/KAIST/dataset_augmentation/train_set/normal -type f | wc -l

# Data Load

In [25]:
trainGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'train_set'),
    target_size=(IMAGE_SIZE, IMAGE_SIZE)
)

validationGen = imageGenerator.flow_from_directory(
    os.path.join('/content/gdrive/MyDrive/KAIST/dataset', 'test_set'),
    target_size=(IMAGE_SIZE, IMAGE_SIZE)
)

Found 81440 images belonging to 2 classes.
Found 9600 images belonging to 2 classes.


# Model setting

In [26]:
for i, l in enumerate(baseModel.layers):
  print(i, l.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [28]:
from keras.models import Model
from tensorflow.keras.layers import Flatten, Dropout, Conv2D, MaxPooling2D, Dense, BatchNormalization

#x = baseModel.output
#x = MaxPooling2D(pool_size=(2, 2))(x)
#x = Flatten()(x)
#x = Dropout(0.5)(x)
#x = Dense(512, activation='relu', input_dim=INPUT_SHAPE)(x)
#x = BatchNormalization()(x)
#x = Dense(256, activation ='relu')(x)
#x = BatchNormalization()(x)
#x = Dense(NUM_CLASSES, activation='softmax')(x)

#model = Model(
#    inputs=baseModel.input
#    , outputs=x
#    )

model = Sequential()

model.add(baseModel)
model.add(Dense(NUM_CLASSES, activation='softmax'))
#model.layers[0].trainable = False

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


# Finetuning

In [29]:
NUM_FREEZE = 150

for l in baseModel.layers:
  l.trainable = False

for l in baseModel.layers[NUM_FREEZE:]:
  l.trainable = True


In [30]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['acc'],
)

In [ ]:
#epochs = 32
#history = model.fit_generator(
#    trainGen, 
#    epochs=epochs,
#    steps_per_epoch=trainGen.samples / epochs, 
#    validation_data=validationGen,
#    validation_steps=trainGen.samples / epochs,
#)
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

reduce_lr = ReduceLROnPlateau(patience=8,verbose=1)
early_stop = EarlyStopping(patience=10,verbose=2,monitor='loss')

epochs = 32
history = model.fit_generator(
    trainGen,  
    epochs=epochs,
    steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
    validation_steps=validationGen.samples / epochs,
    callbacks=[reduce_lr,early_stop],
    workers=1000,
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/32
2545/2545 [==============================] - 1057s 406ms/step - loss: 0.6505 - acc: 0.6225 - val_loss: 21.3058 - val_acc: 0.1902
Epoch 2/32
2545/2545 [==============================] - 944s 369ms/step - loss: 0.6239 - acc: 0.6476 - val_loss: 0.9280 - val_acc: 0.4338
Epoch 3/32
2545/2545 [==============================] - 942s 369ms/step - loss: 0.6125 - acc: 0.6621 - val_loss: 0.6640 - val_acc: 0.7624
Epoch 4/32
2545/2545 [==============================] - 944s 369ms/step - loss: 0.6098 - acc: 0.6651 - val_loss: 3.0893 - val_acc: 0.1886
Epoch 5/32
2545/2545 [==============================] - 958s 375ms/step - loss: 0.6022 - acc: 0.6730 - val_loss: 1.0314 - val_acc: 0.8087
Epoch 6/32
2545/2545 [==============================] - 963s 377ms/step - loss: 0.5989 - acc: 0.6762 - val_loss: 0.9049 - val_acc: 0.1622
Epoch 7/32
2545/2545 [==============================] - 965s 377ms/step - loss: 0.5981 - acc: 0.6784 - val_loss: 4.9156 - val_acc: 0.1896
Epoch 8/32
2545/2545 [==========

In [ ]:
#model.save_weights("/content/gdrive/MyDrive/KAIST/SEP592/checkpoints/resnet50_brca_ckpt")
!mkdir -p WEIGHT_DIR_PATH
WEIGHT_FILE_PATH = '210616_Resnet50_brca_aug_freeze150.h5'

model.save_weights('%s/%s' % (WEIGHT_DIR_PATH, WEIGHT_FILE_PATH))

# `assert_consumed` can be used as validation that all variable values have been
# restored from the checkpoint. See `tf.train.Checkpoint.restore` for other
# methods in the Status object.
#load_status.assert_consumed()

#학습 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

def show_graph(history_dict):
    accuracy = history_dict['acc']
    val_accuracy = history_dict['val_acc']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(16, 1))
    
    plt.subplot(121)
    plt.subplots_adjust(top=2)
    plt.plot(epochs, accuracy, 'ro', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
    plt.title('Trainging and validation accuracy and loss')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy and Loss')

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
              fancybox=True, shadow=True, ncol=5)
#     plt.legend(bbox_to_anchor=(1, -0.1))

    plt.subplot(122)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=5)
#     plt.legend(bbox_to_anchor=(1, 0))

    plt.show()
show_graph(history.history)


In [ ]:
def smooth_curve(points, factor=.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points
smooth_data = {}
for key, val in history.history.items():
    smooth_data[key] = smooth_curve(val)
show_graph(smooth_data)


#모델 평가

In [ ]:
#!ls -l /content/gdrive/MyDrive/KAIST/SEP592/checkpoints | grep resnet50 | grep brca

In [ ]:
#load_status = model.load_weights('%s/%s' % (WEIGHT_DIR_PATH, '210608_0741_resnet50_brca_12_300_300_aug.h5'))

In [ ]:
#model.compile(
#    optimizer='adam',
#    loss='binary_crossentropy', 
#    metrics=['acc'],
#)

In [ ]:
testGenerator = ImageDataGenerator(
    rescale=1./255
)

testGen = testGenerator.flow_from_directory(
    os.path.join('/content/gdrive/MyDrive/KAIST/dataset', 'test_set'),
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle = False,  
)

In [ ]:
#model.evaluate_generator(testGen)
testevaluation = model.evaluate_generator(testGen, verbose=1, steps=testGen.samples//32, workers = 300)

In [ ]:
testevaluation

In [ ]:
#Predicting the classes of images
testprediction = model.predict_generator(testGen, verbose=1, steps=testGen.samples//32)

In [ ]:
print(testGen.class_indices)
print(testprediction)

In [ ]:
testGen.classes

In [ ]:
testprediction[:,0]

In [ ]:
testprediction[:,1]

In [ ]:
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt

nn_fpr, nn_tpr, nn_th = roc_curve(testGen.classes, testprediction[:,0])
auc_pred = auc(nn_fpr, nn_tpr)

plt.figure()
plt.plot(nn_fpr, nn_tpr, label='ROC curve of label "normal"(area = %0.2f)' % auc_pred)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt

nn_fpr, nn_tpr, nn_th = roc_curve(testGen.classes, testprediction[:,1])
auc_pred = auc(nn_fpr, nn_tpr)

plt.figure()
plt.plot(nn_fpr, nn_tpr, label='ROC curve of label "lym"(area = %0.2f)' % auc_pred)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
len(testGen.labels)
#for i in range(100):

# answer가 0인데 predict가 1이면 failCntArr[0]++
failCntArr = [0, 0]
allCntArr = [0, 0]
for i in range(len(testprediction)):
  prdIdx = 1 if testprediction[i][0] < testprediction[i][1] else 0
  ansIdx = testGen.labels[i]
  allCntArr[ansIdx] += 1

  if prdIdx == ansIdx : continue
  #print('[FAIL] predict[%d], answer[%d]' % (prdIdx, ansIdx))
  failCntArr[ansIdx] += 1

print('Pass cnt : lym[%d/%d], normal[%d/%d]' % (failCntArr[0], allCntArr[0], failCntArr[1], allCntArr[1]))